In [3]:
import numpy as np
import pandas as pd

In [4]:
# Load all the training data

client = pd.read_csv(r'C:\Users\Inigo\OneDrive - University of Bristol\Coding Stuff\Applied ML\AppliedML2024\FinalProject\predict-energy-behavior-of-prosumers\client.csv')
forecast = pd.read_csv(r'C:\Users\Inigo\OneDrive - University of Bristol\Coding Stuff\Applied ML\AppliedML2024\FinalProject\predict-energy-behavior-of-prosumers\forecast_weather.csv')
electricity = pd.read_csv(r'C:\Users\Inigo\OneDrive - University of Bristol\Coding Stuff\Applied ML\AppliedML2024\FinalProject\predict-energy-behavior-of-prosumers\electricity_prices.csv')
gas = pd.read_csv(r'C:\Users\Inigo\OneDrive - University of Bristol\Coding Stuff\Applied ML\AppliedML2024\FinalProject\predict-energy-behavior-of-prosumers\gas_prices.csv')
historical = pd.read_csv(r'C:\Users\Inigo\OneDrive - University of Bristol\Coding Stuff\Applied ML\AppliedML2024\FinalProject\predict-energy-behavior-of-prosumers\historical_weather.csv')
train = pd.read_csv(r'C:\Users\Inigo\OneDrive - University of Bristol\Coding Stuff\Applied ML\AppliedML2024\FinalProject\predict-energy-behavior-of-prosumers\train.csv')
weather_station = pd.read_csv(r'C:\Users\Inigo\OneDrive - University of Bristol\Coding Stuff\Applied ML\AppliedML2024\FinalProject\predict-energy-behavior-of-prosumers\weather_station_to_county_mapping.csv')

In [7]:
def generate_features(
    df_data,
    df_client,
    df_gas_prices,
    df_electricity_prices,
    df_forecast_weather,
    df_historical_weather,
    df_weather_station_to_county_mapping,
    train_start_timestep = '2021-09-01 11:00:00',
    gas_end_date = '2022-01-10 23:00:00',
    drop_na = True,  # CHANGE TO FALSE TO KEEP NA VALUES
    drop_datetime = True # CHANGE TO FALSE TO KEEP DATETIME OBJECT
    
    ):

    df_weather_station_to_county_mapping = df_weather_station_to_county_mapping[df_weather_station_to_county_mapping.notnull().all(axis=1)].sort_values(by="county")
    result_dict = dict(zip(zip(round(df_weather_station_to_county_mapping['latitude'],1), round(df_weather_station_to_county_mapping['longitude'],1)), df_weather_station_to_county_mapping['county']))

    df_historical_weather = df_historical_weather[df_historical_weather['datetime'] >= train_start_timestep]

    
    #Merge df_data and df_client
    df_client_try = df_client.copy(deep=True)
    df_client_try['date'] = pd.to_datetime(df_client_try['date'])
    df_client_try['datetime'] = df_client_try['date'].apply(lambda x: [x + pd.Timedelta(hours=i) for i in range(24)])
    df_client_try = df_client_try.explode('datetime')
    df_client_try = df_client_try.drop(['date','data_block_id'], axis=1)
    
    df_data['datetime'] = pd.to_datetime(df_data['datetime'])
    df_data = df_data.merge(df_client_try, on=['county','product_type','is_business','datetime'], how='left')

    endDate = df_client_try['datetime'].max()
    df_data = df_data[df_data['datetime'] <= endDate]
    

    #Merge df_data and df_gas_prices
    df_gas_try = df_gas_prices.copy(deep=True)
    df_gas_try['forecast_date'] = pd.to_datetime(df_gas_try['forecast_date'])
    df_gas_try['datetime'] = df_gas_try['forecast_date'].apply(lambda x: [x + pd.Timedelta(hours=i) for i in range(24)])
    df_gas_try = df_gas_try.explode('datetime')
    df_gas_try = df_gas_try.drop(['forecast_date' , 'origin_date' , 'data_block_id'], axis=1)
    gas_end_date = df_gas_try['datetime'].max()
    df_data = df_data[df_data['datetime'] <= gas_end_date]
    df_data = df_data.merge(df_gas_try, on=['datetime'], how='left')

    #Merge df_data and df_electricity_prices
    df_electricity_prices_try = df_electricity_prices.copy(deep=True)
    df_electricity_prices_try.drop(columns=["origin_date" , 'data_block_id'], inplace=True)
    df_electricity_prices_try['forecast_date'] = pd.to_datetime(df_electricity_prices_try['forecast_date'])
    df_electricity_prices_try.rename(columns={"forecast_date": "datetime"}, inplace=True)
    df_data = df_data.merge(df_electricity_prices_try, on=['datetime'], how='left')

    #Merge df_data and df_forecast_weather
    df_forecast_weather_copy = df_forecast_weather.copy(deep=True)
    df_forecast_weather_copy['county'] = [result_dict.get((x, y), -1) for x, y in zip(df_forecast_weather_copy['latitude'], df_forecast_weather_copy['longitude'])]
    df_forecast_weather_copy = df_forecast_weather_copy[df_forecast_weather_copy['county']!=-1]
    df_forecast_weather_copy['origin_datetime'] = pd.to_datetime(df_forecast_weather_copy['origin_datetime'])
    df_forecast_weather_copy['origin_datetime'] = pd.to_datetime(df_forecast_weather_copy['origin_datetime'].dt.date.astype(str) + ' 02:00:00')
    df_forecast_weather_copy['forecast_datetime'] = df_forecast_weather_copy['origin_datetime'] + pd.to_timedelta(df_forecast_weather_copy['hours_ahead'], unit='h')
    df_forecast_weather_copy = df_forecast_weather_copy.drop('origin_datetime',axis=1)
    df_forecast_weather_copy = df_forecast_weather_copy.sort_values(by=['latitude','longitude','forecast_datetime', 'hours_ahead'])
    df_forecast_weather_copy['cumcount'] = (df_forecast_weather_copy['hours_ahead']-1)//24+1
    
    columns_to_average = [col for col in df_forecast_weather_copy.columns if col not in ['latitude', 'longitude', 'hours_ahead' , 'forecast_datetime','cumcount']]
    agg_dict = {col: 'mean' for col in columns_to_average}
    agg_dict['cumcount'] = 'first'  # to preserve the cumcount value
    df_forecast_weather_copy = df_forecast_weather_copy.groupby(['county','forecast_datetime', 'cumcount']).agg(agg_dict)
    df_forecast_weather_copy=df_forecast_weather_copy.unstack(level=-1)
    df_forecast_weather_copy.columns = [f'{col[0]}_{col[1]}' for col in df_forecast_weather_copy.columns]
    df_forecast_weather_copy.reset_index(inplace=True)
    df_forecast_weather_copy.rename(columns={'forecast_datetime': 'datetime'}, inplace=True)
    df_forecast_weather_copy = df_forecast_weather_copy.drop(['county_1','county_2','cumcount_1','cumcount_2'],axis=1)
    df_forecast_weather_copy.fillna(0, inplace=True)

    df_forecast_weather_copy = df_forecast_weather_copy.add_suffix('_fore') # THIS IS NEW
    
    df_data = pd.merge(df_data, df_forecast_weather_copy, left_on=['county','datetime'], right_on=['county_fore', 'datetime_fore'], how='left')


    #Merge df_data and df_historical_weather
    df_historical_weather_copy = df_historical_weather.copy(deep=True)
    df_historical_weather_copy['datetime'] = pd.to_datetime(df_historical_weather_copy['datetime'])
    df_historical_weather_copy['county'] = [result_dict.get((x, y), -1) for x, y in zip(df_historical_weather_copy['latitude'], df_historical_weather_copy['longitude'])]
    df_historical_weather_copy = df_historical_weather_copy[df_historical_weather_copy['county']!=-1]
    df_historical_weather_copy = df_historical_weather_copy.sort_values(by=['latitude','longitude','datetime'])
    columns_to_average = [col for col in df_historical_weather_copy.columns if col not in ['latitude', 'longitude', 'datetime','county','data_block_id']]
    agg_dict = {col: 'mean' for col in columns_to_average}
    df_historical_weather_copy = df_historical_weather_copy.groupby(['county','datetime']).agg(agg_dict)
    df_historical_weather_copy.reset_index(inplace=True)

    df_historical_weather_copy = df_historical_weather_copy.add_suffix('_hist') # THIS IS NEW

    df_data = pd.merge(df_data, df_historical_weather_copy, left_on=['county','datetime'], right_on=['county_hist', 'datetime_hist'], how='left')

    # Final cleanup

    if drop_na:
        df_data = df_data.dropna()

    df_data = df_data.drop("datetime_fore", axis=1 )
    df_data = df_data.drop("county_fore", axis=1 )
    df_data = df_data.drop("datetime_hist", axis=1 )
    df_data = df_data.drop("county_hist", axis=1 )

    df_data["datetime"] = pd.to_datetime(df_data["datetime"])
    df_data['year'] = df_data['datetime'].dt.year
    df_data['month'] = df_data['datetime'].dt.month
    df_data['day'] = df_data['datetime'].dt.day
    df_data['hour'] = df_data['datetime'].dt.hour
    df_data['weekday'] = df_data['datetime'].dt.weekday

    if drop_datetime:
        df_data = df_data.drop(['datetime'], axis=1)

    return df_data

In [8]:
data_full = generate_features(train, client, gas, electricity, forecast, historical, weather_station, drop_na = True, drop_datetime=True)

In [9]:
# Now download to a csv file (Warning is large so may take a while 1GB)

data_full.to_csv(r'C:\Users\Inigo\OneDrive - University of Bristol\Coding Stuff\Applied ML\AppliedML2024\FinalProject\predict-energy-behavior-of-prosumers\train_full.csv', index=False)